In [3]:
import pandas as pd
import glob
import os

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import numpy as np
import datetime as dt

import json

# Set Connection

In [4]:
#database_path = ""

#engine = create_engine(f"sqlite:///{database_path}")
#con = engine.connect()

# Target Cities

In [6]:
cities = pd.read_csv('Resources/most_populated_cities.csv')

In [7]:
cities=cities.rename(columns={"name":"City", "state":"state_name"})
cities.head()

,City,population,state_name
0,Phoenix,1733626,AZ
1,San Antonio,1581727,TX
2,San Diego,1427719,CA
3,Austin,1011786,TX
4,Fort Worth,942323,TX


In [8]:
cities.dtypes

City          object
population     int64
state_name    object
dtype: object

# Lat and Long

In [9]:
cities2 = pd.read_csv('Resources/uscities.csv')
cities2=cities2.rename(columns={"city":"City"})
cities2


,City,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28333,Gross,Gross,NE,Nebraska,31015,Boyd,42.9461,-98.5697,2,6,polygon,False,True,America/Chicago,3,68719,1840011032
28334,Lotsee,Lotsee,OK,Oklahoma,40143,Tulsa,36.1334,-96.2091,2,39,polygon,False,True,America/Chicago,3,74063,1840021674
28335,The Ranch,The Ranch,MN,Minnesota,27087,Mahnomen,47.3198,-95.6952,2,2,polygon,False,True,America/Chicago,3,56557,1840039629
28336,Shamrock,Shamrock,OK,Oklahoma,40037,Creek,35.9113,-96.5772,2,2,polygon,False,True,America/Chicago,3,74068,1840022701


In [10]:
#merging two city dfs to get all info we want on just our target cities 
city_df = cities2.merge(cities, how = 'inner', right_on = ['City','state_name'], left_on = ['City','state_id'])
city_df.head()

,City,city_ascii,state_id,state_name_x,county_fips,county_name,lat,lng,population_x,density,source,military,incorporated,timezone,ranking,zips,id,population_y,state_name_y
0,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149,478251,FL
1,Phoenix,Phoenix,AZ,Arizona,4013,Maricopa,33.5722,-112.0891,4219697,1253,polygon,False,True,America/Phoenix,1,85008 85009 85006 85007 85004 85083 85086 8508...,1840020568,1733626,AZ
2,Seattle,Seattle,WA,Washington,53033,King,47.6211,-122.3244,3789215,3469,polygon,False,True,America/Los_Angeles,1,98109 98108 98104 98107 98106 98101 98103 9810...,1840021117,776555,WA
3,San Diego,San Diego,CA,California,6073,San Diego,32.8312,-117.1225,3220118,1686,polygon,False,True,America/Los_Angeles,1,92109 92108 92103 92111 92154 92110 92115 9214...,1840021990,1427719,CA
4,Denver,Denver,CO,Colorado,8031,Denver,39.7621,-104.8759,2876625,1831,polygon,False,True,America/Denver,1,80264 80230 80231 80236 80237 80235 80238 8023...,1840018789,749103,CO


In [11]:
#tidying up before exporting to sql
city_df=city_df.drop(columns=['city_ascii','state_name_x','source', 'military','incorporated', 'ranking', 'population_y','state_name_y'])
city_df=city_df.rename(columns={"state_id":"state", "population_x":"population", 'zips': 'zip_codes', 'city': "City"})
city_df


,City,state,county_fips,county_name,lat,lng,population,density,timezone,zip_codes,id
0,Miami,FL,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,America/New_York,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
1,Phoenix,AZ,4013,Maricopa,33.5722,-112.0891,4219697,1253,America/Phoenix,85008 85009 85006 85007 85004 85083 85086 8508...,1840020568
2,Seattle,WA,53033,King,47.6211,-122.3244,3789215,3469,America/Los_Angeles,98109 98108 98104 98107 98106 98101 98103 9810...,1840021117
3,San Diego,CA,6073,San Diego,32.8312,-117.1225,3220118,1686,America/Los_Angeles,92109 92108 92103 92111 92154 92110 92115 9214...,1840021990
4,Denver,CO,8031,Denver,39.7621,-104.8759,2876625,1831,America/Denver,80264 80230 80231 80236 80237 80235 80238 8023...,1840018789
5,San Antonio,TX,48029,Bexar,29.4658,-98.5253,2049293,1231,America/Chicago,78112 78259 78258 78252 78251 78250 78257 7825...,1840022220
6,Austin,TX,48453,Travis,30.3004,-97.7522,1687311,1181,America/Chicago,78749 78748 78741 78742 78745 78744 78747 7874...,1840019590
7,Charlotte,NC,37119,Mecklenburg,35.2080,-80.8304,1512923,1113,America/New_York,28269 28262 28105 28280 28282 28209 28208 2820...,1840014557
8,Jacksonville,FL,12031,Duval,30.3322,-81.6749,1181496,470,America/New_York,32224 32225 32226 32227 32220 32221 32222 3222...,1840015031
9,Raleigh,NC,37183,Wake,35.8325,-78.6435,1038738,1254,America/New_York,27617 27616 27615 27614 27613 27612 27610 2760...,1840014497


In [12]:
#city_df.to_sql('cities', con = con)

# Temperature

In [14]:
temps = pd.read_csv('Resources/city_temperature_v2.csv', low_memory=False)
temps

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,2015,49.9
1,Africa,Algeria,NaN,Algiers,1,2,2015,46.6
2,Africa,Algeria,NaN,Algiers,1,3,2015,48.1
3,Africa,Algeria,NaN,Algiers,1,4,2015,49.2
4,Africa,Algeria,NaN,Algiers,1,5,2015,47.6
...,...,...,...,...,...,...,...,...
573521,North America,US,Wyoming,Cheyenne,5,9,2020,42.3
573522,North America,US,Wyoming,Cheyenne,5,10,2020,43.1
573523,North America,US,Wyoming,Cheyenne,5,11,2020,37.8
573524,North America,US,Wyoming,Cheyenne,5,12,2020,41.9


In [15]:
#reduce down to the desired years 
temps=temps.loc[temps['Year'] >= 2019,:]
temps

,Region,Country,State,City,Month,Day,Year,AvgTemperature
1462,Africa,Algeria,NaN,Algiers,1,1,2019,50.6
1463,Africa,Algeria,NaN,Algiers,1,2,2019,49.9
1464,Africa,Algeria,NaN,Algiers,1,3,2019,50.9
1465,Africa,Algeria,NaN,Algiers,1,4,2019,50.6
1466,Africa,Algeria,NaN,Algiers,1,5,2019,49.7
...,...,...,...,...,...,...,...,...
573521,North America,US,Wyoming,Cheyenne,5,9,2020,42.3
573522,North America,US,Wyoming,Cheyenne,5,10,2020,43.1
573523,North America,US,Wyoming,Cheyenne,5,11,2020,37.8
573524,North America,US,Wyoming,Cheyenne,5,12,2020,41.9


In [16]:
#merge the dates into one colum
cols=["Month","Day","Year"]
temps['Date'] = temps[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")

/var/folders/qc/_ykx45d102d0nr51ysx4tb4h0000gn/T/ipykernel_63636/591227486.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps['Date'] = temps[cols].apply(lambda x: '/'.join(x.values.astype(str)), axis="columns")


In [17]:
#transform date from string to date (have to coerce errors b/c somewhere there is a year that is wrong)
temps['Date']= pd.to_datetime(temps['Date'],errors = 'coerce')

/var/folders/qc/_ykx45d102d0nr51ysx4tb4h0000gn/T/ipykernel_63636/810566525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps['Date']= pd.to_datetime(temps['Date'],errors = 'coerce')


In [18]:
#confirm date transform worked
temps.dtypes

Region                    object
Country                   object
State                     object
City                      object
Month                      int64
Day                        int64
Year                       int64
AvgTemperature           float64
Date              datetime64[ns]
dtype: object

In [19]:
#Filter down to US (could and should have done this above but forgot until after all the date wrangling)
us=temps.loc[temps["Country"] == "US",:]
us

,Region,Country,State,City,Month,Day,Year,AvgTemperature,Date
275107,North America,US,Alabama,Birmingham,1,1,2019,60.3,2019-01-01
275108,North America,US,Alabama,Birmingham,1,2,2019,51.1,2019-01-02
275109,North America,US,Alabama,Birmingham,1,3,2019,51.4,2019-01-03
275110,North America,US,Alabama,Birmingham,1,4,2019,54.7,2019-01-04
275111,North America,US,Alabama,Birmingham,1,5,2019,47.5,2019-01-05
...,...,...,...,...,...,...,...,...,...
573521,North America,US,Wyoming,Cheyenne,5,9,2020,42.3,2020-05-09
573522,North America,US,Wyoming,Cheyenne,5,10,2020,43.1,2020-05-10
573523,North America,US,Wyoming,Cheyenne,5,11,2020,37.8,2020-05-11
573524,North America,US,Wyoming,Cheyenne,5,12,2020,41.9,2020-05-12


In [20]:
#drop unneeded columns
us=us.drop(columns=['Month','Day','Year', 'Region', 'Country'])
us

,State,City,AvgTemperature,Date
275107,Alabama,Birmingham,60.3,2019-01-01
275108,Alabama,Birmingham,51.1,2019-01-02
275109,Alabama,Birmingham,51.4,2019-01-03
275110,Alabama,Birmingham,54.7,2019-01-04
275111,Alabama,Birmingham,47.5,2019-01-05
...,...,...,...,...
573521,Wyoming,Cheyenne,42.3,2020-05-09
573522,Wyoming,Cheyenne,43.1,2020-05-10
573523,Wyoming,Cheyenne,37.8,2020-05-11
573524,Wyoming,Cheyenne,41.9,2020-05-12


In [21]:
# Merge cities and temps to get just the cities we care about
ustemps= cities.merge(us, how = 'inner', right_on = 'City', left_on = "City")
ustemps

,City,population,state_name,State,AvgTemperature,Date
0,Phoenix,1733626,AZ,Arizona,42.9,2019-01-01
1,Phoenix,1733626,AZ,Arizona,41.4,2019-01-02
2,Phoenix,1733626,AZ,Arizona,42.7,2019-01-03
3,Phoenix,1733626,AZ,Arizona,47.7,2019-01-04
4,Phoenix,1733626,AZ,Arizona,54.5,2019-01-05
...,...,...,...,...,...,...
4486,Omaha,479978,NE,Nebraska,50.7,2020-05-09
4487,Omaha,479978,NE,Nebraska,47.7,2020-05-10
4488,Omaha,479978,NE,Nebraska,45.5,2020-05-11
4489,Omaha,479978,NE,Nebraska,51.0,2020-05-12


In [26]:
# ustemps.to_sql('Temperatures', con = con, if_exists = 'append', index = False)

# Air Quality


In [28]:
#merge 13 csv's on import
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('Resources', "airqualitydata*.csv"))))
df

,Date,Country,City,Specie,count,min,max,median,variance
0,3/11/22,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
1,3/16/22,RE,Saint-Denis,co,120,0.1,0.1,0.1,0.00
2,3/18/22,RE,Saint-Denis,co,15,0.1,0.1,0.1,0.00
3,1/2/22,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
4,1/19/22,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
...,...,...,...,...,...,...,...,...,...
582601,2019-07-09,HU,Budapest,wind-gust,282,0.1,12.6,4.4,109.64
582602,2019-07-18,HU,Budapest,wind-gust,338,0.1,9.7,3.4,37.78
582603,2019-07-21,HU,Budapest,wind-gust,337,0.1,17.9,3.8,137.24
582604,2019-07-24,HU,Budapest,wind-gust,325,0.2,6.1,2.7,33.09


In [29]:
#dates are different formats for diff years
# change the format to DD-MM-YYYY
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,Country,City,Specie,count,min,max,median,variance
0,2022-03-11,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
1,2022-03-16,RE,Saint-Denis,co,120,0.1,0.1,0.1,0.00
2,2022-03-18,RE,Saint-Denis,co,15,0.1,0.1,0.1,0.00
3,2022-01-02,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
4,2022-01-19,RE,Saint-Denis,co,96,0.1,0.1,0.1,0.00
...,...,...,...,...,...,...,...,...,...
582601,2019-07-09,HU,Budapest,wind-gust,282,0.1,12.6,4.4,109.64
582602,2019-07-18,HU,Budapest,wind-gust,338,0.1,9.7,3.4,37.78
582603,2019-07-21,HU,Budapest,wind-gust,337,0.1,17.9,3.8,137.24
582604,2019-07-24,HU,Budapest,wind-gust,325,0.2,6.1,2.7,33.09


In [30]:
#filter out out just the US
us=df.loc[df["Country"] == "US",:]
us

,Date,Country,City,Specie,count,min,max,median,variance
97972,2022-02-26,US,Oklahoma City,co,22,3.4,6.7,4.5,8.62
97973,2022-03-07,US,Oklahoma City,co,14,1.2,3.4,2.3,5.92
97974,2022-02-16,US,Oklahoma City,co,23,3.4,11.2,6.7,37.72
97975,2022-01-07,US,Oklahoma City,co,27,3.4,8.9,5.6,31.68
97976,2022-03-03,US,Oklahoma City,co,41,2.3,12.3,5.6,67.69
...,...,...,...,...,...,...,...,...,...
186607,2019-07-17,US,Fort Worth,wind-speed,43,0.1,7.2,3.0,27.81
186608,2019-07-22,US,Fort Worth,wind-speed,36,0.6,7.0,3.0,28.88
186609,2019-08-18,US,Fort Worth,wind-speed,39,0.1,5.5,2.2,18.76
186610,2019-09-09,US,Fort Worth,wind-speed,46,0.1,6.0,2.5,17.98


In [31]:
#get a city list- how many of these (57) are on our original list of 14? 12 (no Mesa or Frisco TX)
us.City.unique()

array(['Oklahoma City', 'Raleigh', 'Memphis', 'Jackson', 'Boston',
       'Richmond', 'Portland', 'Boise', 'Austin', 'Honolulu', 'Fresno',
       'Milwaukee', 'Hartford', 'Washington D.C.', 'Chicago', 'Houston',
       'Indianapolis', 'Atlanta', 'Charlotte', 'Sacramento', 'Oakland',
       'Providence', 'Staten Island', 'Brooklyn', 'Springfield',
       'The Bronx', 'San Jose', 'Los Angeles', 'Detroit', 'Little Rock',
       'Baltimore', 'Phoenix', 'Omaha', 'El Paso', 'Dallas', 'Seattle',
       'Manhattan', 'Miami', 'Jacksonville', 'San Antonio',
       'Philadelphia', 'San Diego', 'Columbus', 'Saint Paul', 'Denver',
       'Salt Lake City', 'Albuquerque', 'San Francisco', 'Salem',
       'Madison', 'Nashville', 'Tucson', 'Queens', 'Tallahassee',
       'Fort Worth', 'Las Vegas', 'Columbia'], dtype=object)

In [32]:
#merge with our target cities list to narrow our focus
usair = cities.merge(us, how = 'inner', right_on = 'City', left_on = "City")
usair

,City,population,state_name,Date,Country,Specie,count,min,max,median,variance
0,Phoenix,1733626,AZ,2022-02-14,US,so2,53,0.1,0.6,0.1,0.14
1,Phoenix,1733626,AZ,2022-01-17,US,so2,53,0.1,0.6,0.1,0.14
2,Phoenix,1733626,AZ,2022-01-25,US,so2,16,0.1,0.6,0.1,0.16
3,Phoenix,1733626,AZ,2022-01-30,US,so2,5,0.1,0.1,0.1,0.00
4,Phoenix,1733626,AZ,2022-02-07,US,so2,17,0.1,0.6,0.1,0.28
...,...,...,...,...,...,...,...,...,...,...,...
142046,Miami,478251,FL,2019-09-23,US,pressure,72,1012.0,1016.5,1014.0,12.08
142047,Miami,478251,FL,2019-10-01,US,pressure,72,1010.8,1014.3,1012.4,7.36
142048,Miami,478251,FL,2019-08-06,US,pressure,72,1014.7,1017.4,1015.8,5.15
142049,Miami,478251,FL,2019-08-13,US,pressure,72,1012.2,1016.9,1013.9,11.87


In [33]:
#what measurements do we get?
#use the temp we got above
usair.Specie.unique()

array(['so2', 'no2', 'pressure', 'wind-gust', 'co', 'temperature', 'o3',
       'wind-speed', 'pm10', 'humidity', 'pm25', 'precipitation',
       'wind gust', 'wind speed', 'dew', 'wd'], dtype=object)

In [34]:
#make each Specie their own DF, which becomes its own table (droping colums we dont need for these tables)
usco=usair.loc[usair["Specie"] == 'co',:].drop(columns=['population','state_name','Country'])
usco.head()

,City,Date,Specie,count,min,max,median,variance
306,Phoenix,2022-01-13,co,105,1.2,22.3,7.8,253.11
307,Phoenix,2022-01-29,co,93,1.1,11.2,3.4,88.78
308,Phoenix,2022-02-07,co,86,1.2,13.4,4.5,107.58
309,Phoenix,2022-02-12,co,120,1.1,19.7,7.8,379.86
310,Phoenix,2022-01-10,co,78,1.2,16.7,3.4,258.84


In [35]:
# usco.to_sql('Carbon_Monoxide', con = con, if_exists = 'append', index = False)

In [36]:
ushumid=usair.loc[usair["Specie"] == 'humidity',:].drop(columns=['population','state_name','Country'])
# ushumid.to_sql('Humidity', con = con, if_exists = 'append', index = False)

In [37]:
uspm25=usair.loc[usair["Specie"] == 'pm25',:].drop(columns=['population','state_name','Country'])
# uspm25.to_sql('Particulate_Matter_25', con = con, if_exists = 'append', index = False)

In [38]:
usgust=usair.loc[usair["Specie"] == 'wind-gust',:].drop(columns=['population','state_name','Country'])
# usgust.to_sql('Wind_Gusts', con = con, if_exists = 'append', index = False)

In [39]:
uspm10=usair.loc[usair["Specie"] == 'pm10',:].drop(columns=['population','state_name','Country'])
# uspm10.to_sql('Particulate_Matter_10', con = con, if_exists = 'append', index = False)

In [40]:
usno2=usair.loc[usair["Specie"] == 'no2',:].drop(columns=['population','state_name','Country'])
# usno2.to_sql('Nitrogen_Dioxide', con = con, if_exists = 'append', index = False)

In [41]:
uspressure=usair.loc[usair["Specie"] == 'pressure',:].drop(columns=['population','state_name','Country'])
# uspressure.to_sql('Pressure', con = con, if_exists = 'append', index = False)

In [42]:
usspeed=usair.loc[usair["Specie"] == 'wind-speed',:].drop(columns=['population','state_name','Country'])
# usspeed.to_sql('Wind_Speed', con = con, if_exists = 'append', index = False)

In [43]:
uso3=usair.loc[usair["Specie"] == 'o3',:].drop(columns=['population','state_name','Country'])
# usso3.to_sql('Sulfer_Trioxide', con = con, if_exists = 'append', index = False)

In [44]:
usso2=usair.loc[usair["Specie"] == 'so2',:].drop(columns=['population','state_name','Country'])
# usso2.to_sql('Sulfer_Dioxide', con = con, if_exists = 'append', index = False)

In [45]:
usdew=usair.loc[usair["Specie"] == 'dew',:].drop(columns=['population','state_name','Country'])
# usdew.to_sql('Dew_Point', con = con, if_exists = 'append', index = False)

In [46]:
uswd=usair.loc[usair["Specie"] == 'wd',:].drop(columns=['population','state_name','Country'])
# uswd.to_sql('Wind_Direction', con = con, if_exists = 'append', index = False)

In [47]:
usprecip=usair.loc[usair["Specie"] == 'precipitation',:].drop(columns=['population','state_name','Country'])
# usprecip.to_sql('Precipitation', con = con, if_exists = 'append', index = False)